In [1]:
import altair as alt
import polars as pl

alt.data_transformers.enable("vegafusion")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

Exploratory analysis into across solver actions. Some initial questions are:
* Where is across directing the majority of flow to? 
* Which chains have the highest connectivity?
* Which tokens are being transported the most?

There is some initial results on token volume and by origin chain [here](https://dune.com/risk_labs/across-protocol-stats) and [here](https://dune.com/sandman2797/across-bridge-stats)

In [2]:
network_dict = {
    "name": [
        "Arbitrum", "Arbitrum Nova", "Aurora", "Avalanche", "Base", 
        "Berachain Bartio", "Blast", "Boba", "Bsc", "C1 Milkomeda", 
        "Celo", "Chiliz", "Crab", "Cyber", "Darwinia", "Ethereum Mainnet", 
        "Fantom", "Flare", "Fuji", "Galadrial Devnet", "Gnosis", 
        "Lukso", "Manta", "Mantle", "Merlin", "Metis", "Mev Commit", 
        "Mode", "Moonbeam", "Neon Evm", "Optimism", "Polygon", 
        "Polygon zkEVM", "Rsk", "Saakuru", "Scroll", "Shimmer Evm", 
        "Taiko", "Taiko Jolnr", "X Layer", "Zeta", "Zircuit", 
        "ZKsync", "Zora"
    ],
    "id": [
        42161, 42170, 1313161554, 43114, 8453, 80084, 81457, 288, 56, 2001, 
        42220, 8888, 44, 7560, 46, 1, 250, 14, 43113, 696969, 100, 
        42, 169, 5000, 4200, 1088, 17864, 34443, 1284, 245022934, 10, 137, 
        1101, 30, 7225878, 534352, 148, 167000, 1088, 196, 7000, 
        48900, 324, 7777777
    ]
}



In [8]:
network_names_df = pl.from_dict(network_dict).with_columns(pl.col('id').cast(pl.UInt64))
across_df = pl.read_parquet('data/across/*.parquet').join(network_names_df, left_on='destinationChainId', right_on='id').join(network_names_df, left_on='chain_id', right_on='id', suffix='_origin')

In [9]:
across_df.shape

(189, 31)

In [10]:
across_df.head(5)

destinationChainId,depositId,depositor,inputToken,outputToken,inputAmount,outputAmount,quoteTimestamp,fillDeadline,exclusivityDeadline,recipient,exclusiveRelayer,message,hash,block_number,extra_data,to,from,nonce,type,block_hash,timestamp,base_fee_per_gas,gas_used_block,max_priority_fee_per_gas,max_fee_per_gas,effective_gas_price,gas_used,chain_id,name,name_origin
u64,u64,str,str,str,f64,f64,i64,u64,i64,str,str,str,str,u64,str,str,str,u64,u8,str,u64,f64,u64,f64,f64,f64,f64,u64,str,str
10,2521067,"""0xa1abf02a05276d9df410e7b60f91ed4056116222""","""0xaf88d065e77c8cc2239327c5edb3a432268e5831""","""0x0b2c639c533813f4aa9d7837caf62653d097ff85""",134003350,133996804,1727712642,1727713226,1727712926,"""0xa1abf02a05276d9df410e7b60f91ed4056116222""","""0xd19454648eedd9bee8d75142d503c5ee9179fc70""","""0x""","""0x1e971941a7932176f59a0c39adad2483a6e731069117130bf101b4751a587d4e""",258973702,"""0x6d061aef1ef65018189b588cee34e831319c2fd4239a40a91bb8aae12766d7ec""","""0x6b1e09905e5643680f926c78125d0c15648153ef""","""0xd19454648eedd9bee8d75142d503c5ee9179fc70""",3264,2,"""0xa070792639ab977bc8202aecf5a59c22872993d040201dc699486e78960078eb""",1727712642,10000000,3376454,0,12000000,10000000,773695,42161,"""Optimism""","""Arbitrum"""
10,2521068,"""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0xaf88d065e77c8cc2239327c5edb3a432268e5831""","""0x0000000000000000000000000000000000000000""",10049500,10041310,1727712431,4294967295,0,"""0xd11dd37165bc9520bf94fc5e21206ca5df4636e7""","""0x0000000000000000000000000000000000000000""","""0x""","""0xcea0e0b3ffc421965312e0628706e08395a76c265d1974c90317995721b5fe54""",258973768,"""0x6d061aef1ef65018189b588cee34e831319c2fd4239a40a91bb8aae12766d7ec""","""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0xd11dd37165bc9520bf94fc5e21206ca5df4636e7""",409,2,"""0xc76739398e222e85eab43746687139c59754cb26b3d2c1d35ba48939b43085e0""",1727712659,10000000,3397524,10000000,10000000,10000000,692994,42161,"""Optimism""","""Arbitrum"""
8453,2521069,"""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0x82af49447d8a07e3bd95bd0d56f35241523fbab1""","""0x0000000000000000000000000000000000000000""",32000000000000000,31988880148010176,1727712431,4294967295,0,"""0x4b224244588f8d0a4d205ff6770a29b45a891ba7""","""0x0000000000000000000000000000000000000000""","""0x""","""0x2b005a4f5671a39c77098dab7114f837814394bdcfef7cbb3561b37257841753""",258973813,"""0x6d061aef1ef65018189b588cee34e831319c2fd4239a40a91bb8aae12766d7ec""","""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0x4b224244588f8d0a4d205ff6770a29b45a891ba7""",95,2,"""0x70ff75e74ff498ea18368f12b9a0b00800a8ead458d0703e33c3d29c811009ae""",1727712670,10000000,1961172,0,13500000,10000000,493962,42161,"""Base""","""Arbitrum"""
534352,2521071,"""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0x82af49447d8a07e3bd95bd0d56f35241523fbab1""","""0x0000000000000000000000000000000000000000""",15000000000000000,14931510316242858,1727712431,4294967295,0,"""0x9f86d47cbdf2e4f24c17c398f9ec0690d253a833""","""0x0000000000000000000000000000000000000000""","""0x""","""0x54886b49ecf455fc3569146833444d27abb2c132024de4bd63a03668b179bbfd""",258973882,"""0x6d061aef1ef65018189b588cee34e831319c2fd4239a40a91bb8aae12766d7ec""","""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0x9f86d47cbdf2e4f24c17c398f9ec0690d253a833""",268,2,"""0xbd8d936444e253749215e05695ce6baf4c251d1b8db66d0f26213a73d8e95fcd""",1727712687,10000000,3130186,0,13500000,10000000,495887,42161,"""Scroll""","""Arbitrum"""
10,2521072,"""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0x82af49447d8a07e3bd95bd0d56f35241523fbab1""","""0x0000000000000000000000000000000000000000""",7000000000000000,6996333110478354,1727712431,4294967295,0,"""0x907557f7f3e947219de3749ab66e4e90ac786224""","""0x0000000000000000000000000000000000000000""","""0x""","""0x39c4ded582692c93abc5adfa074507198c8b9501db03ff91a6548cc9c35cfa96""",258974023,"""0x6d061aef1ef65018189b588cee34e831319c2fd4239a40a91bb8aae12766d7ec""","""0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae""","""0

In [6]:
across_df.filter(pl.col('depositor')== '0x0e9b063789909565ceda1fba162474405a151e66')

destinationChainId,depositId,depositor,inputToken,outputToken,inputAmount,outputAmount,quoteTimestamp,fillDeadline,exclusivityDeadline,recipient,exclusiveRelayer,message,hash,block_number,extra_data,to,from,nonce,type,block_hash,timestamp,base_fee_per_gas,gas_used_block,max_priority_fee_per_gas,max_fee_per_gas,effective_gas_price,gas_used,chain_id,name
u64,u64,str,str,str,f64,f64,i64,u64,i64,str,str,str,str,u64,str,str,str,u64,u8,str,u64,f64,u64,f64,f64,f64,f64,u64,str


In [7]:
print(funds_deposited.shape)

NameError: name 'funds_deposited' is not defined

In [6]:
(
    funds_deposited
    .group_by('destinationChainId').agg([
    pl.len().alias('count')
    ])
    
).sort(by='count', descending=True)

destinationChainId,count,name
u64,u32,str
8453,9881,"""Base"""
10,4780,"""Optimism"""
534352,3314,"""Scroll"""
324,2868,"""ZKsync"""
1,1260,"""Ethereum Mainnet"""
137,1077,"""Polygon"""
81457,369,"""Blast"""
34443,322,"""Mode"""
7777777,17,"""Zora"""
